# Лабораторная работа 6. Выбор оптимального классификатора

В этой лабораторной работе вам потребуется выбрать наилучший классификатор с оптимальными параметрами для задачи про пассажиров ["Титаника"](https://ru.wikipedia.org/wiki/Титаник).

__Задание 1.__  
Загрузите данные (см. предыдущую лабораторную работу).

In [162]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

train_data = pd.read_csv('./input/train.csv')
test_data = pd.read_csv('./input/test.csv')

__Задание 2.__  
Проведите предобработку данных (см. предыдущую лабораторную работу).

In [163]:
def process_age(df, cut_points, label_names):
    df['Age_categories']=df['Age']
    for i in range (len(cut_points)-1):
        df['Age']=df['Age'].fillna(-0.5)        
        df['Age_categories'][(cut_points[i]<df['Age']) & (df['Age']<=cut_points[i+1])]=label_names[i]
    return df

def create_dummies(df, column_name):
    enc = OneHotEncoder(handle_unknown='ignore')
    X=np.array(df[column_name]).reshape(len(df[column_name]), 1)
    enc.fit(X)
    bin_cod=enc.transform(X).toarray().T
    
    for i in range (len(np.unique(X))):
        name=column_name+'_'+str(np.unique(X)[i])
        df[name]=pd.Series(bin_cod[i])    
    return df

cut_points = [-1, 0, 5, 12, 18, 35, 60, 100]
label_names = ["Missing", "Infant", "Child", "Teenager", "Young_Adult", "Adult", "Senior"]

train = process_age(train_data, cut_points, label_names)
test = process_age(test_data,cut_points,label_names)

train = create_dummies(train_data, "Pclass")
test = create_dummies(test_data, "Pclass")

train = create_dummies(train_data, "Sex")
test = create_dummies(test_data, "Sex")

train = create_dummies(train_data, "Age_categories")
test = create_dummies(test_data, "Age_categories")
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_categories,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Age_categories_Adult,Age_categories_Child,Age_categories_Infant,Age_categories_Missing,Age_categories_Senior,Age_categories_Teenager,Age_categories_Young_Adult
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Young_Adult,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Young_Adult,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Young_Adult,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Young_Adult,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Young_Adult,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Young_Adult,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,-0.5,1,2,W./C. 6607,23.4500,NaN,S,Missing,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Young_Adult,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


__Задание 3.__  
Примените масштабирование признаков (`StandardScaler`, `MinMaxScaler`).

In [155]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

variable=np.array([test['Fare']]).T
scaler = MinMaxScaler()
scaler.fit(variable)
test['FareScaled']=pd.Series(scaler.transform(variable).T[0])         #test

variable=np.array([train['Fare']]).T
scaler = MinMaxScaler()
scaler.fit(variable)
train['FareScaled']=pd.Series(scaler.transform(variable).T[0])       #train

__Задание 4.__  
Примените различные преобразования признаков (`PolynomialFeatures`).

In [156]:
from sklearn.preprocessing import PolynomialFeatures

variable=train[['FareScaled', 'Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']].to_numpy()
print(variable)
poly = PolynomialFeatures(3)
new_ar=poly.fit_transform(variable)

[[0.01415106 0.         0.         ... 1.         0.         0.        ]
 [0.13913574 1.         0.         ... 0.         1.         0.        ]
 [0.01546857 0.         0.         ... 1.         0.         0.        ]
 ...
 [0.04577135 0.         0.         ... 1.         0.         0.        ]
 [0.0585561  0.         0.         ... 0.         1.         0.        ]
 [0.01512699 0.         0.         ... 1.         0.         0.        ]]


__Задание 5.__  
Обучите несколько классификаторов, в том числе:  
1. Логистическую регрессию (`LogisticRegression`).
1. Метод опорных векторов (`SVC`).
1. Метод *k* ближайших соседей (`KNeighborsClassifier`).
1. Наивный байесовский классификатор (`MultinomialNB`).
1. Деревья решений (`DecisionTreeClassifier`).
1. Случайный лес (`RandomForestClassifier`).
1. AdaBoost (`AdaBoost`).
1. Градиентный бустинг (`GradientBoostingClassifier`).

Для обучения и проверки качества можно использовать функцию `train_test_split()`.

In [157]:
# разбиение выборки

from sklearn.model_selection import train_test_split

X=new_ar
y=np.array(train['Survived'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train:')
print(X_train.shape)
print('\nX_test:')
print(X_test.shape)
print('\ny_train:')
print(y_train.shape)
print('\ny_test:')
print(y_test.shape)

X_train:
(712, 560)

X_test:
(179, 560)

y_train:
(712,)

y_test:
(179,)


In [158]:
clfs=[LogisticRegression(), SVC(gamma='auto'), KNeighborsClassifier(n_neighbors=5), MultinomialNB(), 
      DecisionTreeClassifier(random_state=0), RandomForestClassifier(max_depth=5, random_state=0),
      AdaBoostClassifier(n_estimators=100, random_state=0), 
      GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)]
for classif in clfs:
    clf=classif
    clf.fit(X_train, y_train)
    print('\n')
    print(classif)
    print('Accuracy (test):')
    print(clf.score(X_test, y_test))



LogisticRegression()
Accuracy (test):
0.8212290502793296


SVC(gamma='auto')
Accuracy (test):
0.7821229050279329


KNeighborsClassifier()
Accuracy (test):
0.8379888268156425


MultinomialNB()
Accuracy (test):
0.8044692737430168


DecisionTreeClassifier(random_state=0)
Accuracy (test):
0.8324022346368715


RandomForestClassifier(max_depth=5, random_state=0)
Accuracy (test):
0.8156424581005587


AdaBoostClassifier(n_estimators=100, random_state=0)
Accuracy (test):
0.8044692737430168


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)
Accuracy (test):
0.8156424581005587


In [159]:
# без фич и масштаб-я
X=train[['FareScaled', 'Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']]
y=np.array(train['Survived'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [160]:
clfs=[LogisticRegression(), SVC(gamma='auto'), KNeighborsClassifier(n_neighbors=5), MultinomialNB(), 
      DecisionTreeClassifier(random_state=0), RandomForestClassifier(max_depth=5, random_state=0),
      AdaBoostClassifier(n_estimators=100, random_state=0), 
      GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)]
for classif in clfs:
    clf=classif
    clf.fit(X_train, y_train)
    print('\n')
    print(classif)
    print('Accuracy (test):')
    print(clf.score(X_test, y_test))



LogisticRegression()
Accuracy (test):
0.8044692737430168


SVC(gamma='auto')
Accuracy (test):
0.7877094972067039


KNeighborsClassifier()
Accuracy (test):
0.8379888268156425


MultinomialNB()
Accuracy (test):
0.770949720670391


DecisionTreeClassifier(random_state=0)
Accuracy (test):
0.8156424581005587


RandomForestClassifier(max_depth=5, random_state=0)
Accuracy (test):
0.8100558659217877


AdaBoostClassifier(n_estimators=100, random_state=0)
Accuracy (test):
0.7932960893854749


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)
Accuracy (test):
0.8100558659217877


__Задание 6.__  
При помощи `Pipeline` и `GridSearchCV` выберите оптимальную архитектуру:
1. Метод масштабирования.
1. Степень полинома в `PolynomialFeatures`.
1. Параметры классификаторов (в том числе, параметры регуляризации).

Заносите в таблицу Excel результаты тестирования (варианты параметров, оценки качества).

In [134]:
X=train[['FareScaled', 'Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2', 'Fare']]
y=np.array(train['Survived'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
# Логистическая регрессия

parameters_grid = {
       'linear_model__C' : np.arange(1, 2, 0.5), # параметр регуляризации
       'preprocessing__degree': np.arange(1, 3),
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('linear_model', LogisticRegression())])
#print(classifier.get_params())
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('linear_model', LogisticRegression())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'linear_model__C': 1.0, 'preprocessing__degree': 1}
0.7879084967320261

MinMaxScaler
{'linear_model__C': 1.5, 'preprocessing__degree': 1}
0.8101307189542485


In [136]:
# K-MEANS

import sklearn.model_selection as model_selection
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

parameters_grid = {
       'neighbors__n_neighbors' : np.arange(1, 20),
       'preprocessing__degree': np.arange(1, 3),
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('neighbors', KNeighborsClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('neighbors', KNeighborsClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'neighbors__n_neighbors': 6, 'preprocessing__degree': 1}
0.7882352941176469

MinMaxScaler
{'neighbors__n_neighbors': 10, 'preprocessing__degree': 1}
0.7879084967320261


In [137]:
# Метод опорных векторов

parameters_grid = {
       'svm__C' : np.arange(1, 2, 0.2), # параметр регуляризации
       'svm__gamma' : [ 'auto'],#'scale',
       'svm__kernel' : [ 'poly', 'linear', 'sigmoid','rbf'],
       'svm__degree': np.arange(1, 5),      
       'preprocessing__degree': np.arange(1, 3),
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('svm', SVC())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('svm', SVC())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'preprocessing__degree': 1, 'svm__C': 1.0, 'svm__degree': 1, 'svm__gamma': 'auto', 'svm__kernel': 'linear'}
0.8101307189542484

MinMaxScaler
{'preprocessing__degree': 1, 'svm__C': 1.2, 'svm__degree': 1, 'svm__gamma': 'auto', 'svm__kernel': 'linear'}
0.8156862745098039


In [138]:
# Наивный байесовский классификатор

parameters_grid = {
       'naive_bayes__alpha' : np.arange(1, 3, 0.2),        
       'preprocessing__degree': np.arange(1, 3),
   }

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('naive_bayes', MultinomialNB())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


MinMaxScaler
{'naive_bayes__alpha': 1.0, 'preprocessing__degree': 2}
0.7549019607843137


In [145]:
# Деревья решений

parameters_grid = {           
       'preprocessing__degree': np.arange(1,3),
       'decisions__criterion' : ['gini', 'entropy'], 
        'decisions__splitter': ['best', 'random'],
        'decisions__min_samples_split' : np.arange(1, 5, 1),
        'decisions__min_samples_leaf' : np.arange(1, 5, 1),
        'decisions__max_features' : ['auto', 'sqrt','log2'],# 
        'decisions__min_impurity_decrease': np.arange(0, 1, 0.2), 
   }


classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('decisions', DecisionTreeClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('decisions', DecisionTreeClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'preprocessing__degree': 2}
0.7437908496732026

MinMaxScaler
{'preprocessing__degree': 2}
0.7326797385620913


In [126]:
# Случайный лес

parameters_grid = {
       'ensemble__max_depth': np.arange(1, 7),       
       'preprocessing__degree': np.arange(1, 3), 
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', RandomForestClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', RandomForestClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'ensemble__max_depth': 2, 'preprocessing__degree': 2}
0.7771241830065359

MinMaxScaler
{'ensemble__max_depth': 5, 'preprocessing__degree': 2}
0.7771241830065359


In [140]:
# AdaBoost 

parameters_grid = {    
       'preprocessing__degree': np.arange(1, 4),
       'ensemble__n_estimators' : np.arange(95, 105),
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', AdaBoostClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', AdaBoostClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'ensemble__n_estimators': 100, 'preprocessing__degree': 1}
0.7601307189542483

MinMaxScaler
{'ensemble__n_estimators': 100, 'preprocessing__degree': 1}
0.7601307189542483


In [141]:
# Градиентный бустинг

parameters_grid = {    
       'preprocessing__degree': np.arange(1, 3),
       'ensemble__n_estimators' : np.arange(95, 105),
       'ensemble__learning_rate' : np.arange(0, 1, 0.1),
       'ensemble__max_depth' : np.arange(1, 10),
   }

classifier = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', GradientBoostingClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nStandardScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)

classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures()), ('ensemble', GradientBoostingClassifier())])
cv = StratifiedKFold(n_splits=10, random_state=0)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)
grid_cv.fit(X_test, y_test)
print('\nMinMaxScaler')
print(grid_cv.best_params_)
print(grid_cv.best_score_)


StandardScaler
{'ensemble__learning_rate': 0.9, 'ensemble__max_depth': 2, 'ensemble__n_estimators': 103, 'preprocessing__degree': 1}
0.7937908496732027

MinMaxScaler
{'ensemble__learning_rate': 0.9, 'ensemble__max_depth': 2, 'ensemble__n_estimators': 95, 'preprocessing__degree': 1}
0.7937908496732027


__Задание 7.__  
1. Выберите несколько лучших классификаторов (от 3 до 10).
1. Обучите выбранные классификаторы на всех доступных размеченных данных.
1. Получите результаты предсказания для тестовых данных.
1. Отправьте результаты на сервер [Kaggle](https://ru.wikipedia.org/wiki/Титаник).

## GradientBoostingClassifier

In [171]:
X=np.array(train[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])
y=np.array(train['Survived'])
X_test=np.array(test[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])

      
poly = PolynomialFeatures(3)
X=poly.fit_transform(X)
X_test=poly.fit_transform(X_test)

clf=GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(X, y)
predictions=clf.predict(X_test)

test_ids = test["PassengerId"]
submission_df = {"PassengerId": test_ids,
                 "Survived": predictions}
submission = pd.DataFrame(submission_df)

submission.to_csv('output/GradientBoostingClassifier(3d).csv', index=False)

## AdaBoostClassifier

In [170]:
X=np.array(train[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])
y=np.array(train['Survived'])
X_test=np.array(test[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])

      
poly = PolynomialFeatures(3)
X=poly.fit_transform(X)
X_test=poly.fit_transform(X_test)

clf=AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X, y)
predictions=clf.predict(X_test)

test_ids = test["PassengerId"]
submission_df = {"PassengerId": test_ids,
                 "Survived": predictions}
submission = pd.DataFrame(submission_df)

submission.to_csv('output/AdaBoostClassifier(3d).csv', index=False)

### DecisionTreeClassifier

In [42]:
X=np.array(train[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])
y=np.array(train['Survived'])
X_test=np.array(test[['Age_categories_Adult', 'Age_categories_Child', 'Age_categories_Infant', 'Age_categories_Missing', 'Age_categories_Senior', 'Age_categories_Teenager', 'Age_categories_Young_Adult',
                  'Sex_female', 'Sex_male', 'Pclass_3', 'Pclass_1', 'Pclass_2']])

In [45]:
classifier = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures(2)), 
                       ('decisions', DecisionTreeClassifier(criterion='entropy', max_features='sqrt', min_impurity_decrease=0,
                                                           min_samples_leaf = 3, min_samples_split = 3, splitter = 'random'))])
classifier.fit(X, y)
predictions=classifier.predict(X_test)

In [46]:
test_ids = test["PassengerId"]
submission_df = {"PassengerId": test_ids,
                 "Survived": predictions}
submission = pd.DataFrame(submission_df)

In [47]:
submission.to_csv('output/DecisionTreeClassifier_titanic_submission.csv', index=False)

### KNeighborsClassifier

In [48]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X, y)
predictions=classifier.predict(X_test)

In [49]:
test_ids = test["PassengerId"]
submission_df = {"PassengerId": test_ids,
                 "Survived": predictions}
submission = pd.DataFrame(submission_df)

In [50]:
submission.to_csv('output/KNeighborsClassifier_titanic_submission.csv', index=False)

### RandomForestClassifier 

In [51]:
clf = RandomForestClassifier(max_depth=5, random_state=30)
clf.fit(X, y)
predictions=classifier.predict(X_test)

In [52]:
test_ids = test["PassengerId"]
submission_df = {"PassengerId": test_ids,
                 "Survived": predictions}
submission = pd.DataFrame(submission_df)

In [53]:
submission.to_csv('output/RandomForestClassifier_titanic_submission.csv', index=False)